In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Thu Aug 22 2019 

CPython 3.6.8
IPython 7.6.0

numpy 1.16.4
scipy 1.2.1
sklearn 0.21.2
pandas 0.24.2

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-58-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nolds
import data
import mne
from random import randint

from config import *
from data.utils import prepare_dfs, prepare_resp_non, prepare_dep_non, get_metapkl
from data.data_files import files_builder, DataKind
from classification.prediction import predict
from classification.scorers import scorer_factory
    
metapkl = get_metapkl()
meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)
data = np.transpose(files_builder(DataKind('processed')).single_file('1a.fif').df.values)

Opening raw data file /home/miroslav/Source/thesis_project/data/processed/1a.fif...
This filename (/home/miroslav/Source/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 19104 =      0.000 ...    76.416 secs
Ready.


/home/miroslav/Source/thesis_project/src/data/utils.py:18: RuntimeWarning: This filename (/home/miroslav/Source/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(file_path)


In [3]:
def ff(row, col, t1, t2=0):
    if row[col] <= t1:
        return -1
    elif row[col] <= t2: 
        return 0
    else:
        return 1
    
metapkl['resp'] = metapkl.apply(lambda row: ff(row, 'change', 1.8), axis=1)
metapkl = metapkl.astype({'resp': 'category'})                                           
print(metapkl.loc[(slice(None), 'a'), 'resp'].value_counts())

-1    70
 1    63
Name: resp, dtype: int64


In [4]:
from itertools import combinations
from sklearn import svm, datasets, metrics
from sklearn.feature_selection import (SelectFromModel, RFE, RFECV, SelectKBest, mutual_info_classif, chi2)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from genetic_selection import GeneticSelectionCV
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

estimators = {
    svm.SVC(kernel='linear', class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
        'kernel': ('linear', 'poly', 'rbf'),
        'gamma': ('auto', 'scale'),
        # 'decision_function_shape' : ('ovo', 'ovr'),
    },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}

def get_selectors(estimator):
    return {
        'RFECV': RFECV(estimator, 5), 
        'SelectFromModel': SelectFromModel(estimator), 
        # 'SelectKBest': SelectKBest(chi2, 3),
        'Genetic': GeneticSelectionCV(estimator,
                                      cv=5,
                                      verbose=0,
                                      scoring=scorer,
                                      n_population=80,
                                      crossover_proba=0.8,
                                      mutation_proba=0.2,
                                      n_generations=80,
                                      crossover_independent_proba=0.5,
                                      mutation_independent_proba=0.05,
                                      tournament_size=5,
                                      caching=True,
                                      n_jobs=1
                                     ),
    }

scorer = scorer_factory(metrics.roc_auc_score, average='weighted')
features = ('corr', 'lyap', 'sampen', 'dfa', 'hurst', 'higu')
comb_size = 1
grid_search_cv = 5

for estimator, params in estimators.items():
    for cols in combinations(features, comb_size):
        for selector_name, selector in get_selectors(estimator).items():
            print(cols)
            print(selector_name)
            gs = GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=grid_search_cv)
            predict('resp', 'a', cols, estimator, metapkl, gs=gs, show_selected=True, selector=selector)
            print(gs.best_params_)
            print('\n\n')

/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


('corr',)
RFECV
[('P4', 'corr'), ('O2', 'corr'), ('T4', 'corr'), ('Cz', 'corr')]
Class distribution:  {-1: 64, 1: 56}
Accuracy: 0.62 (+/- 0.07)
Precision: 0.65 (+/- 0.07)
Recall: 0.62 (+/- 0.07)
F1: 0.60 (+/- 0.08)
0.62 $\pm$ 0.07 & 0.65 $\pm$ 0.07 & 0.62 $\pm$ 0.07 & 0.60 $\pm$ 0.08 &  \\ \hline
{'C': 1.5, 'gamma': 'auto', 'kernel': 'linear'}



('corr',)
SelectFromModel
[('FP1', 'corr'), ('C4', 'corr'), ('P3', 'corr'), ('P4', 'corr'), ('O2', 'corr'), ('F8', 'corr'), ('T3', 'corr'), ('T4', 'corr'), ('T5', 'corr'), ('Cz', 'corr')]
Class distribution:  {-1: 64, 1: 56}
Accuracy: 0.62 (+/- 0.09)
Precision: 0.67 (+/- 0.09)
Recall: 0.62 (+/- 0.09)
F1: 0.61 (+/- 0.10)
0.62 $\pm$ 0.09 & 0.67 $\pm$ 0.09 & 0.62 $\pm$ 0.09 & 0.61 $\pm$ 0.10 &  \\ \hline
{'C': 0.5, 'gamma': 'auto', 'kernel': 'linear'}



('corr',)
Genetic
[('FP1', 'corr'), ('FP2', 'corr'), ('P3', 'corr'), ('P4', 'corr'), ('O1', 'corr'), ('T3', 'corr'), ('T5', 'corr'), ('T6', 'corr'), ('Cz', 'corr')]
Class distribution:  {-1: 64, 

/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

# CROSS-VALIDATED

In [7]:
estimators = [
    # LogisticRegression(C=1, penalty='l1', class_weight='balanced'),
    svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]

# 75
channels = [('FP2', 'lyap'), ('F3', 'lyap'), ('O1', 'lyap'), ('T4', 'lyap'), ('T6', 'lyap'), 
            ('F3', 'sampen'), ('C3', 'sampen'), ('T6', 'sampen')]
# channels = [('F3', 'lyap'), ('O2', 'lyap'), ('T5', 'lyap'), ('T6', 'lyap'), ('FP2', 'corr'),
#             ('F4', 'corr'), ('O2', 'corr')]

for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.75 (+/- 0.10)
Precision: 0.77 (+/- 0.09)
Recall: 0.75 (+/- 0.10)
F1: 0.75 (+/- 0.10)
0.75 $\pm$ 0.10 & 0.77 $\pm$ 0.09 & 0.75 $\pm$ 0.10 & 0.75 $\pm$ 0.10 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [53]:
estimators = [
    # LogisticRegression(class_weight='balanced'),
    svm.SVC(C=1.5, class_weight='balanced', kernel='linear'),
]
# 71
# channels = [('F3', 'lyap'), ('C4', 'lyap'), ('O1', 'lyap'), ('F7', 'lyap'), ('T3', 'lyap'), ('T6', 'lyap'), ]
channels = [('F3', 'lyap'), ('F4', 'lyap'), ('T5', 'lyap'), ('T6', 'lyap')]

for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.71 (+/- 0.08)
Precision: 0.73 (+/- 0.08)
Recall: 0.71 (+/- 0.08)
F1: 0.70 (+/- 0.09)
0.71  $\pm$ 0.08  & 0.73  $\pm$ 0.08  & 0.71  $\pm$ 0.08  & 0.70  $\pm$ 0.09  &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
estimators = [
    LogisticRegression(C=1, penalty='l2', class_weight='balanced'),
    # svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]
# 71
# channels = [('F3', 'sampen'), ('C4', 'sampen'), ('C3', 'sampen'), ('Fz', 'sampen')]
channels = [('FP1', 'sampen'), ('F3', 'sampen'), ('P3', 'sampen'), ('Cz', 'sampen')]

for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.66 (+/- 0.09)
Precision: 0.66 (+/- 0.09)
Recall: 0.66 (+/- 0.09)
F1: 0.65 (+/- 0.10)
0.66 $\pm$ 0.09 & 0.66 $\pm$ 0.09 & 0.66 $\pm$ 0.09 & 0.65 $\pm$ 0.10 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [9]:
estimators = [
    LogisticRegression(C=1, penalty='l2', class_weight='balanced'),
    # svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]

channels = [('F3', 'higu'), ('F8', 'higu')]


for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.66 (+/- 0.05)
Precision: 0.72 (+/- 0.08)
Recall: 0.66 (+/- 0.05)
F1: 0.64 (+/- 0.05)
0.66 $\pm$ 0.05 & 0.72 $\pm$ 0.08 & 0.66 $\pm$ 0.05 & 0.64 $\pm$ 0.05 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [12]:
estimators = [
    # LogisticRegression(C=1, penalty='l2', class_weight='balanced'),
    svm.SVC(C=2, class_weight='balanced', kernel='rbf'),
]

channels = [('C3', 'hurst'), ('T6', 'hurst')]


for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.63 (+/- 0.09)
Precision: 0.64 (+/- 0.10)
Recall: 0.63 (+/- 0.09)
F1: 0.62 (+/- 0.09)
0.63 $\pm$ 0.09 & 0.64 $\pm$ 0.10 & 0.63 $\pm$ 0.09 & 0.62 $\pm$ 0.09 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [8]:
estimators = [
    LogisticRegression(C=1, penalty='l2', class_weight='balanced'),
    # svm.SVC(C=2, class_weight='balanced', kernel='rbf'),
]

channels = [('F3', 'corr'), ('F4', 'corr'), ('O2', 'corr'), ('Pz', 'corr')]


for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.67 (+/- 0.09)
Precision: 0.70 (+/- 0.11)
Recall: 0.67 (+/- 0.09)
F1: 0.65 (+/- 0.10)
0.67 $\pm$ 0.09 & 0.70 $\pm$ 0.11 & 0.67 $\pm$ 0.09 & 0.65 $\pm$ 0.10 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [11]:
estimators = [
    # LogisticRegression(C=1, penalty='l2', class_weight='balanced'),
    svm.SVC(C=10, class_weight='balanced', kernel='linear'),
]

channels = [('T3', 'dfa'), ('T4', 'dfa'), ('Cz', 'dfa')]

for estimator in estimators:
    est = predict('resp', 'a', None, estimator, channels=channels)

Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.64 (+/- 0.15)
Precision: 0.65 (+/- 0.15)
Recall: 0.64 (+/- 0.15)
F1: 0.63 (+/- 0.15)
0.64 $\pm$ 0.15 & 0.65 $\pm$ 0.15 & 0.64 $\pm$ 0.15 & 0.63 $\pm$ 0.15 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [40]:
channels = [('F3', 'lyap'), ('F4', 'lyap'), ('C4', 'lyap'), ('P3', 'lyap'), ('P4', 'lyap'), ('F8', 'lyap'), 
            ('T4', 'lyap'), ('T5', 'lyap'), ('T6', 'lyap'), ('Fz', 'lyap'),]
channels = [('F3', 'sampen'), ('C4', 'sampen'), ('C3', 'sampen'), ('Fz', 'sampen')]
channels = [('F3', 'higu'), ('P4', 'higu'), ('C3', 'higu'), ('Fz', 'higu'), ('F8', 'higu')]
channels = [('C4', 'hurst'), ('T5', 'hurst')]

estimators = {
    svm.SVC(kernel='linear', class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
        'kernel': ('linear', 'poly', 'rbf'),
        # 'decision_function_shape' : ('ovo', 'ovr'),
    },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}

for estimator, params in estimators.items():
    gs = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=5)
    selector = GeneticSelectionCV(estimator,
                                      cv=5,
                                      verbose=0,
                                      scoring=scorer,
                                      n_population=80,
                                      crossover_proba=0.8,
                                      mutation_proba=0.2,
                                      n_generations=80,
                                      crossover_independent_proba=0.5,
                                      mutation_independent_proba=0.05,
                                      tournament_size=5,
                                      caching=True,
                                      n_jobs=-1)
    selector = None
    est = predict('resp', 'a', None, estimator, show_selected=True, channels=channels, gs=gs, selector=selector)
    print(gs.best_params_)

/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


Class distribution:  {-1: 47, 1: 47}
Accuracy: 0.63 (+/- 0.11)
Precision: 0.66 (+/- 0.13)
Recall: 0.63 (+/- 0.11)
F1: 0.61 (+/- 0.11)


IndexError: tuple index out of range

# RESULTS FOR FIRST AND LAST 15% QUANTILE OF CHANGE (34 / 32)

## Lyapunov exponent

In [18]:
estimator = LogisticRegression(C=3.5, penalty='l1', class_weight='balanced')
predict('resp', None, ('lyap',), estimator,
        channels=('FP2', 'F3', 'F4', 'C3', 'P3', 'P4', 'F7', 'F8', 'T6', 'Fz', 'Cz'))

Training distribution:  {-1: 24, 1: 22}
Testing distribution:  {-1: 10, 1: 10}
0.80 & 0.80 & 0.80 & $\left( \begin{smallmatrix} 9 & 1 \\ 3 & 7 \end{smallmatrix} \right)$ & FP2, F3, F4, C3, P3, P4, F7, F8, T6, Fz, Cz \\ \hline
Accuracy: 0.64 (+/- 0.19)


LogisticRegression(C=3.5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## DFA

In [17]:
estimator = LogisticRegression(C=50, penalty='l2', class_weight='balanced')
predict('resp', None, ('dfa',), estimator, channels=('FP2', 'F3', 'O1', 'T5', 'T6', 'Cz'))

Training distribution:  {-1: 24, 1: 22}
Testing distribution:  {-1: 10, 1: 10}
0.80 & 0.80 & 0.80 & $\left( \begin{smallmatrix} 8 & 2 \\ 2 & 8 \end{smallmatrix} \right)$ & FP2, F3, O1, T5, T6, Cz \\ \hline
Accuracy: 0.67 (+/- 0.14)


LogisticRegression(C=50, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Hurst exponent

In [16]:
estimator = LogisticRegression(C=3, penalty='l2', class_weight='balanced')
predict('resp', None, ('hurst',), estimator,
        channels=('FP2', 'F3', 'O1', 'T5', 'T6', 'Cz'))

Training distribution:  {-1: 24, 1: 22}
Testing distribution:  {-1: 10, 1: 10}
0.70 & 0.70 & 0.70 & $\left( \begin{smallmatrix} 7 & 3 \\ 3 & 7 \end{smallmatrix} \right)$ & FP2, F3, O1, T5, T6, Cz \\ \hline
Accuracy: 0.60 (+/- 0.18)


LogisticRegression(C=3, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Higuchi dimension
Results around 55%.

## Correlation dimension

In [15]:
estimator = svm.SVC(C=2, kernel='rbf', class_weight='balanced', decision_function_shape='ovo')
predict('resp', None, ('corr'), estimator, 
                   channels=('F4', 'C4', 'O1', 'F7', 'F8', 'T5', 'T6'))

Training distribution:  {-1: 24, 1: 22}
Testing distribution:  {-1: 10, 1: 10}
1.00 & 1.00 & 1.00 & $\left( \begin{smallmatrix} 10 & 0 \\ 0 & 10 \end{smallmatrix} \right)$ & F4, C4, O1, F7, F8, T5, T6 \\ \hline
Accuracy: 0.65 (+/- 0.21)


SVC(C=2, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# RESULTS FOR FIRST AND LAST TERCILE OF CHANGE (74 / 66)

## Lyapunov exponent

In [178]:
estimator = svm.SVC(C=1, kernel='rbf', class_weight='balanced', decision_function_shape='ovo')
predict('resp', None, ('lyap'), estimator, channels=('F3', 'F4', 'O1', 'O2', 'T6', 'Fz'))

Training distribution:  {-1: 51, 1: 47}
Testing distribution:  {-1: 23, 1: 19}
0.76 & 0.76 & 0.77 & $\left( \begin{smallmatrix} 16 & 7 \\ 3 & 16 \end{smallmatrix} \right)$ & F3, F4, O1, O2, T6, Fz \\ \hline


SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## DFA

In [177]:
estimator = LogisticRegression(class_weight='balanced')
params = {'C': [0.5, 1.0, 1.5, 2, 2.5, 3, 3.5],
        'penalty': ['l2', 'l1'],}
estimator = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer)
predict('resp', None, ('dfa',), estimator, channels=('C3', 'P4', 'F7', 'T6'))
estimator.best_params_

Training distribution:  {-1: 51, 1: 47}
Testing distribution:  {-1: 23, 1: 19}
0.64 & 0.64 & 0.64 & $\left( \begin{smallmatrix} 15 & 8 \\ 7 & 12 \end{smallmatrix} \right)$ & C3, P4, F7, T6 \\ \hline


{'C': 1.0, 'penalty': 'l2'}

### Alpha / Theta envelope
All estimators fail (50/50). Very similar values between groupds and small variance. There is still chance for depression score, however.

Same for sample entropy.

## Hurst exponent

In [174]:
estimator = LogisticRegression(C=3, penalty='l2', class_weight='balanced')
predict('resp', None, ('hurst',), estimator, channels=('P3', 'F7', 'T4', 'T6', 'Cz'))

Training distribution:  {-1: 51, 1: 47}
Testing distribution:  {-1: 23, 1: 19}
0.71 & 0.71 & 0.73 & $\left( \begin{smallmatrix} 14 & 9 \\ 3 & 16 \end{smallmatrix} \right)$ & P3, F7, T4, T6, Cz \\ \hline


LogisticRegression(C=3, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Higuchi dimension

In [176]:
estimator = LogisticRegression(C=3.5, penalty='l1', class_weight='balanced')
predict('resp', None, ('higu',), estimator, channels=('FP2',))

Training distribution:  {-1: 51, 1: 47}
Testing distribution:  {-1: 23, 1: 19}
0.69 & 0.69 & 0.70 & $\left( \begin{smallmatrix} 13 & 10 \\ 3 & 16 \end{smallmatrix} \right)$ & FP2 \\ \hline


LogisticRegression(C=3.5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Correlation dimension

In [175]:
estimator = LogisticRegression(C=3.5, penalty='l1', class_weight='balanced')
estimator = predict('resp', None, ('corr',), estimator, channels=('F4', 'C4', 'T5'))

Training distribution:  {-1: 51, 1: 47}
Testing distribution:  {-1: 23, 1: 19}
0.69 & 0.69 & 0.70 & $\left( \begin{smallmatrix} 14 & 9 \\ 4 & 15 \end{smallmatrix} \right)$ & F4, C4, T5 \\ \hline


In [166]:
estimator = svm.SVC(C=2, kernel='rbf', class_weight='balanced', decision_function_shape='ovo')
predict('resp', None, ('corr'), estimator, channels=('F4', 'C4', 'O1', 'F7', 'F8', 'T5'))

Training distribution:  {-1: 51, 1: 47}
Testing distribution:  {-1: 23, 1: 19}
Accuracy score: 0.67
Confusion matrix:
 [[13 10]
 [ 4 15]]
Precision score:  0.6901960784313724
Recall score:  0.6666666666666666
f1 score:  0.6643939393939394
ROC AUC score:  0.6773455377574371


SVC(C=2, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Manual selection

## LLE

In [36]:
from random import randint
estimators = {
    svm.SVC(kernel='linear', class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
        'kernel': ('linear', 'poly', 'rbf'),
    },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
seed = randint(0, 100000)
print('Seed: %s' % seed)
channels = ['Fz']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('resp', None, ('lyap',), est, show_selected=True, channels=channels, seed=seed)

Seed: 5870
Training distribution:  {-1: 29, 1: 27}
Testing distribution:  {-1: 11, 1: 14}
Accuracy score: 0.68
Confusion matrix:
 [[9 2]
 [6 8]]
Precision score:  0.7120000000000001
Recall score:  0.68
f1 score:  0.6779487179487179
ROC AUC score:  0.6948051948051948
Training distribution:  {-1: 29, 1: 27}
Testing distribution:  {-1: 11, 1: 14}
Accuracy score: 0.72
Confusion matrix:
 [[9 2]
 [5 9]]
Precision score:  0.741038961038961
Recall score:  0.72
f1 score:  0.7200000000000002
ROC AUC score:  0.7305194805194805


In [38]:
seed = randint(0, 100000)
print('Seed: %s' % seed)
channels = ['P3']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('resp', None, ('lyap',), est, show_selected=True, channels=channels, seed=seed)

Seed: 91461
Training distribution:  {-1: 25, 1: 31}
Testing distribution:  {-1: 15, 1: 10}
Accuracy score: 0.72
Confusion matrix:
 [[9 6]
 [1 9]]
Precision score:  0.78
Recall score:  0.72
f1 score:  0.72
ROC AUC score:  0.75
Training distribution:  {-1: 25, 1: 31}
Testing distribution:  {-1: 15, 1: 10}
Accuracy score: 0.56
Confusion matrix:
 [[13  2]
 [ 9  1]]
Precision score:  0.4878787878787878
Recall score:  0.56
f1 score:  0.48316008316008324
ROC AUC score:  0.4833333333333334


## Higuchi fractal dimension

In [49]:
estimators = {
    # svm.SVC(kernel='linear', class_weight='balanced'): {
    #     'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
    #     'kernel': ('linear', 'poly', 'rbf'),
    # },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
# 71875
seed = randint(0, 100000)
print('Seed: %s' % seed)
channels = ['FP2', 'Cz']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('resp', None, ('higu',), est, show_selected=True, channels=channels, seed=seed)

Seed: 95528
Training distribution:  {-1: 25, 1: 31}
Testing distribution:  {-1: 15, 1: 10}
Accuracy score: 0.88
Confusion matrix:
 [[12  3]
 [ 0 10]]
Precision score:  0.9076923076923077
Recall score:  0.88
f1 score:  0.8811594202898552
ROC AUC score:  0.9
